In [36]:
# importamos librerias que vamos a necesitar
import pandas as pd 
import ast
import json
from textblob import TextBlob

Se descomprimen cada uno de los archivos previo a importarlos con pandas, quedamos a disposicion de su archivo json con el cual estaremos realizando el proceso de ETL

In [37]:
# lista vacia donde ingresaremos los datos del archivo
lista_vacia=[]
# abrimos archivo json
with open('data/australian_user_reviews.json','r',encoding='utf-8') as archivo:
    # creamos un bucle for para iterar e ingresar los datos dentro de la lista vacia
    for line in archivo.readlines():
        lista_vacia.append(ast.literal_eval(line))

# creamos un dataframe con los datos ingresados en la lista
user_reviews= pd.DataFrame(lista_vacia)
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [38]:
# informacion general del dataframe
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [39]:
# data de la columna 'user_id'
user_reviews['user_id']

0        76561197970982479
1                  js41637
2                evcentric
3                    doctr
4                maplemage
               ...        
25794    76561198306599751
25795             Ghoustik
25796    76561198310819422
25797    76561198312638244
25798          LydiaMorley
Name: user_id, Length: 25799, dtype: object

In [40]:
#contamos valores nulos de la columna 'user_id'
nulos_user_id= user_reviews['user_id'].isnull().sum()
print(f'valores nulos columna user_id: {nulos_user_id}')

valores nulos columna user_id: 0


In [41]:
#data de la columna 'user_url'
user_reviews['user_url']

0        http://steamcommunity.com/profiles/76561197970...
1                     http://steamcommunity.com/id/js41637
2                   http://steamcommunity.com/id/evcentric
3                       http://steamcommunity.com/id/doctr
4                   http://steamcommunity.com/id/maplemage
                               ...                        
25794    http://steamcommunity.com/profiles/76561198306...
25795                http://steamcommunity.com/id/Ghoustik
25796    http://steamcommunity.com/profiles/76561198310...
25797    http://steamcommunity.com/profiles/76561198312...
25798             http://steamcommunity.com/id/LydiaMorley
Name: user_url, Length: 25799, dtype: object

In [42]:
#contamos valores nulos de la columna 'user_url'
nulos_user_url= user_reviews['user_url'].isnull().sum()
print(f'valores nulos columna user_url: {nulos_user_url}')

valores nulos columna user_url: 0


In [43]:
# eliminamos columnas 'user_url' ya que no la necesitaremos
user_reviews= user_reviews.drop(columns=['user_url'], axis=1)

In [44]:
# data de la columna 'reviews' que vamos a desanidar
user_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

# transformacion

desanidamos los datos de la columna reviews

In [45]:
#desanidamos la columna 'reviews' por medio del metodo explode
metodo_explode = user_reviews.explode('reviews') 
#en el proceso de desanidar sus datos se extraen a nuevas columnas, a la vez que la columna 'reviews' es eliminada
user_reviews = pd.concat([metodo_explode.drop(['reviews'], axis=1), metodo_explode['reviews'].apply(pd.Series)], axis=1)
user_reviews = user_reviews.drop_duplicates(subset='review', keep='first')

In [46]:
user_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,,Posted July 10.,,130,No ratings yet,True,if you liked Half life i would really recommen...,NaN
25797,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN


In [47]:
# debemos eliminar la ultima columna ya que solo contiene valores nulos
nulos_0= user_reviews[0].isnull().sum()
print(f'cantidad valores nulos columna 0: {nulos_0}')

cantidad valores nulos columna 0: 55314


In [48]:
# eliminamos la ultima columna
user_reviews= user_reviews.drop(columns=[0], axis=1)

In [49]:
# funcion para contar la cantidad de celdas vacias en las columnas 'funny' y 'last_edited'
def contador_celdas_vacias(dataframe,columna):
    contador=dataframe[columna].apply(lambda x: x == '').sum()
    return contador

# aplicamos funcion en la columna 'funny'
celdas_vacias_funny= contador_celdas_vacias(user_reviews,'funny')
print(f'cantidad celdas vacias columna funny: {celdas_vacias_funny}')
# aplicamos funcion en la columna 'last_edited'
celdas_vacias_last_edited= contador_celdas_vacias(user_reviews,'last_edited')
print(f'cantidad celdas vacias columna last_edited: {celdas_vacias_last_edited}')

cantidad celdas vacias columna funny: 47555
cantidad celdas vacias columna last_edited: 49368


In [50]:
# eliminamos las columnas 'funny' y 'last_edited' ya que no tienen muchas celdas vacias y su informacion no es util para el proyecto
user_reviews= user_reviews.drop(columns=['funny', 'last_edited'], axis=1)
user_reviews.head()

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [51]:
# data de la columna 'posted'
user_reviews['posted'].sample(20)

25610    Posted September 15, 2014.
5342       Posted October 20, 2015.
10165     Posted February 14, 2014.
23628        Posted March 31, 2014.
187            Posted May 11, 2015.
8077           Posted May 23, 2015.
1592            Posted September 2.
20569     Posted November 10, 2015.
1082      Posted September 9, 2014.
25715            Posted February 3.
4183      Posted February 23, 2014.
21607      Posted January 30, 2015.
14574      Posted December 1, 2015.
13810         Posted July 13, 2015.
18408     Posted February 14, 2015.
14487       Posted August 29, 2015.
11497       Posted January 8, 2015.
14740      Posted January 25, 2014.
1603       Posted October 24, 2012.
24665     Posted December 12, 2013.
Name: posted, dtype: object

# transformacion

extraemos el año de la columna posted para posteriormente almacenarlo en una nueva columna llamada year

In [52]:
# extraemos el año de la columna 'posted' en una nueva columna llamada 'year'
user_reviews['year']=user_reviews['posted'].str.extract(r'(\d{4})')
# eliminamos columna posted
user_reviews=user_reviews.drop(columns=['posted'],axis=1)

In [53]:
# observamos la media para calcular valores faltantes de la columna 'year'
user_reviews['year'].value_counts(normalize=True)

year
2014    0.449068
2015    0.376969
2013    0.136931
2012    0.024506
2011    0.011085
2010    0.001440
Name: proportion, dtype: float64

In [54]:
# con la mediana rellenamos datos faltantes de la columna
user_reviews['year'].fillna(user_reviews['year'].median(), inplace=True)

In [55]:
# comprobamos nulos
nulos_year=user_reviews['year'].isnull().sum()
print(f'cantidad valores nulos columna year: {nulos_year}')

cantidad valores nulos columna year: 0


In [56]:
# convertimos tipo de dato de columna 'year' a int
user_reviews['year']=user_reviews['year'].astype(int)

In [57]:
# eliminamo columna ya que su informacion es innecesaria para nuestro proceso de analisis
user_reviews=user_reviews.drop(columns=['helpful'], axis=1)

chequeo de valores nulos

In [58]:
# contamos valores nulos en el dataframe
user_reviews.isnull().sum()

user_id      0
item_id      1
recommend    1
review       1
year         0
dtype: int64

In [59]:
# eliminamos los valores nulos de estas columnas ya que solo son 1 registro nulo por columna
user_reviews= user_reviews.dropna(subset=['item_id', 'recommend', 'review'])

In [60]:
# verificamos valores nulos
user_reviews.isnull().sum()

user_id      0
item_id      0
recommend    0
review       0
year         0
dtype: int64

# analisis de sentimiento y creacion de la columna sentiment_analysis

In [61]:
# creamos la columna 'sentiment_analysis' por medio de una funcion y la aplicamos sobre la columna 'review'
from textblob import TextBlob

# funcion donde realizaremos el analisis de sentimiento
def analyze_sentiment(review):
    if pd.isnull(review):  # resenia es nulo devuelve 1 (neutral)
        return 1
    else:
        analysis = TextBlob(review)
        # asignamos valores de acuerdo con la polaridad del análisis de sentimiento
        if analysis.sentiment.polarity < 0:
            return 0  # Sentimiento negativo
        elif analysis.sentiment.polarity == 0:
            return 1  # Sentimiento neutro
        else:
            return 2  # Sentimiento positivo
        
user_reviews['sentiment_analysis']= user_reviews['review'].apply(analyze_sentiment)

In [62]:
# cantidad de cada valor en la columna 'sentiment_analysis'
user_reviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    31127
0    12381
1    11805
Name: count, dtype: int64

In [63]:
user_reviews

,user_id,item_id,recommend,review,year,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2
0,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
1,js41637,251610,True,I know what you think when you see this title ...,2014,2
1,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,0
...,...,...,...,...,...,...
25797,76561198312638244,130,True,if you liked Half life i would really recommen...,2014,2
25797,76561198312638244,70,True,a must have classic from steam definitely wort...,2014,2
25797,76561198312638244,362890,True,this game is a perfect remake of the original ...,2014,2
25798,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2014,2


In [64]:
#informacion general del dataframe
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55313 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             55313 non-null  object
 1   item_id             55313 non-null  object
 2   recommend           55313 non-null  object
 3   review              55313 non-null  object
 4   year                55313 non-null  int32 
 5   sentiment_analysis  55313 non-null  int64 
dtypes: int32(1), int64(1), object(4)
memory usage: 2.7+ MB


In [65]:
#cambiamos tipo de dato de la columna 'item_id'
user_reviews['item_id']=user_reviews['item_id'].astype(int)

In [66]:
# cambiamos tipo de dato de la columna 'recommend'
user_reviews['recommend']= user_reviews['recommend'].astype(bool)

In [67]:
#informacion general del dataframe
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55313 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             55313 non-null  object
 1   item_id             55313 non-null  int32 
 2   recommend           55313 non-null  bool  
 3   review              55313 non-null  object
 4   year                55313 non-null  int32 
 5   sentiment_analysis  55313 non-null  int64 
dtypes: bool(1), int32(2), int64(1), object(2)
memory usage: 2.2+ MB


chequeo de valores nulos

In [68]:
#verificamos cantidad de valores nulos
user_reviews.isnull().sum()

user_id               0
item_id               0
recommend             0
review                0
year                  0
sentiment_analysis    0
dtype: int64

In [69]:
# reiniciamos los indices
user_reviews.reset_index(drop=True, inplace=True)

In [70]:
user_reviews

,user_id,item_id,recommend,review,year,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
3,js41637,251610,True,I know what you think when you see this title ...,2014,2
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,0
...,...,...,...,...,...,...
55308,76561198312638244,130,True,if you liked Half life i would really recommen...,2014,2
55309,76561198312638244,70,True,a must have classic from steam definitely wort...,2014,2
55310,76561198312638244,362890,True,this game is a perfect remake of the original ...,2014,2
55311,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2014,2


# guardamos archivo a clean_user_reviews.parquet

In [71]:
ruta= 'data/clean_user_reviews.parquet'

user_reviews.to_parquet(ruta, index=False)